In [28]:
import argparse
import os
import pprint

import gymnasium as gym
import numpy as np
import torch
from gymnasium.spaces import Box
from torch.utils.tensorboard import SummaryWriter

from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.policy import ICMPolicy, PPOPolicy
from tianshou.trainer import OnpolicyTrainer
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import MLP, ActorCritic, Net
from tianshou.utils.net.discrete import Actor, Critic, IntrinsicCuriosityModule

In [29]:

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="LunarLander-v2")
    parser.add_argument("--reward-threshold", type=float, default=200)
    parser.add_argument("--seed", type=int, default=1626)
    parser.add_argument("--buffer-size", type=int, default=20000)
    parser.add_argument("--lr", type=float, default=3e-4)
    parser.add_argument("--gamma", type=float, default=0.99)
    parser.add_argument("--epoch", type=int, default=10)
    parser.add_argument("--step-per-epoch", type=int, default=50000)
    parser.add_argument("--step-per-collect", type=int, default=2000)
    parser.add_argument("--repeat-per-collect", type=int, default=10)
    parser.add_argument("--batch-size", type=int, default=64)
    parser.add_argument("--hidden-sizes", type=int, nargs="*", default=[64, 64])
    parser.add_argument("--training-num", type=int, default=20)
    parser.add_argument("--test-num", type=int, default=100)
    parser.add_argument("--logdir", type=str, default="log")
    parser.add_argument("--render", type=float, default=0.0)
    parser.add_argument(
        "--device",
        type=str,
        default="cuda" if torch.cuda.is_available() else "cpu",
    )
    # ppo special
    parser.add_argument("--vf-coef", type=float, default=0.5)
    parser.add_argument("--ent-coef", type=float, default=0.0)
    parser.add_argument("--eps-clip", type=float, default=0.2)
    parser.add_argument("--max-grad-norm", type=float, default=0.5)
    parser.add_argument("--gae-lambda", type=float, default=0.95)
    parser.add_argument("--rew-norm", type=int, default=0)
    parser.add_argument("--norm-adv", type=int, default=0)
    parser.add_argument("--recompute-adv", type=int, default=0)
    parser.add_argument("--dual-clip", type=float, default=None)
    parser.add_argument("--value-clip", type=int, default=0)
    parser.add_argument(
        "--lr-scale",
        type=float,
        default=1.0,
        help="use intrinsic curiosity module with this lr scale",
    )
    parser.add_argument(
        "--reward-scale",
        type=float,
        default=0.01,
        help="scaling factor for intrinsic curiosity reward",
    )
    parser.add_argument(
        "--forward-loss-weight",
        type=float,
        default=0.2,
        help="weight for the forward model loss in ICM",
    )
    return parser.parse_known_args()[0]

In [30]:
args=get_args()
env = gym.make(args.task,render_mode="human")
args.state_shape = env.observation_space.shape or env.observation_space.n
args.action_shape = env.action_space.shape or env.action_space.n
if args.reward_threshold is None:
    default_reward_threshold = {"CartPole-v0": 200}
    args.reward_threshold = default_reward_threshold.get(args.task, env.spec.reward_threshold)
    

In [31]:
# train_envs = gym.make(args.task)
# you can also use tianshou.env.SubprocVectorEnv
train_envs = DummyVectorEnv([lambda: gym.make(args.task) for _ in range(args.training_num)])
# test_envs = gym.make(args.task)
test_envs = DummyVectorEnv([lambda: gym.make(args.task) for _ in range(args.test_num)])

In [32]:


# seed
np.random.seed(args.seed)
torch.manual_seed(args.seed)
train_envs.seed(args.seed)
test_envs.seed(args.seed)
# model

[[1626],
 [1627],
 [1628],
 [1629],
 [1630],
 [1631],
 [1632],
 [1633],
 [1634],
 [1635],
 [1636],
 [1637],
 [1638],
 [1639],
 [1640],
 [1641],
 [1642],
 [1643],
 [1644],
 [1645],
 [1646],
 [1647],
 [1648],
 [1649],
 [1650],
 [1651],
 [1652],
 [1653],
 [1654],
 [1655],
 [1656],
 [1657],
 [1658],
 [1659],
 [1660],
 [1661],
 [1662],
 [1663],
 [1664],
 [1665],
 [1666],
 [1667],
 [1668],
 [1669],
 [1670],
 [1671],
 [1672],
 [1673],
 [1674],
 [1675],
 [1676],
 [1677],
 [1678],
 [1679],
 [1680],
 [1681],
 [1682],
 [1683],
 [1684],
 [1685],
 [1686],
 [1687],
 [1688],
 [1689],
 [1690],
 [1691],
 [1692],
 [1693],
 [1694],
 [1695],
 [1696],
 [1697],
 [1698],
 [1699],
 [1700],
 [1701],
 [1702],
 [1703],
 [1704],
 [1705],
 [1706],
 [1707],
 [1708],
 [1709],
 [1710],
 [1711],
 [1712],
 [1713],
 [1714],
 [1715],
 [1716],
 [1717],
 [1718],
 [1719],
 [1720],
 [1721],
 [1722],
 [1723],
 [1724],
 [1725]]

In [33]:

net = Net(args.state_shape, hidden_sizes=args.hidden_sizes, device=args.device)
actor = Actor(net, args.action_shape, device=args.device).to(args.device)
critic = Critic(net, device=args.device).to(args.device)
actor_critic = ActorCritic(actor, critic)

In [34]:

# orthogonal initialization
for m in actor_critic.modules():
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.orthogonal_(m.weight)
        torch.nn.init.zeros_(m.bias)
optim = torch.optim.Adam(actor_critic.parameters(), lr=args.lr)
dist = torch.distributions.Categorical
policy = PPOPolicy(
    actor=actor,
    critic=critic,
    optim=optim,
    dist_fn=dist,
    action_scaling=isinstance(env.action_space, Box),
    discount_factor=args.gamma,
    max_grad_norm=args.max_grad_norm,
    eps_clip=args.eps_clip,
    vf_coef=args.vf_coef,
    ent_coef=args.ent_coef,
    gae_lambda=args.gae_lambda,
    reward_normalization=args.rew_norm,
    dual_clip=args.dual_clip,
    value_clip=args.value_clip,
    action_space=env.action_space,
    deterministic_eval=True,
    advantage_normalization=args.norm_adv,
    recompute_advantage=args.recompute_adv,
)

In [35]:

feature_dim = args.hidden_sizes[-1]
feature_net = MLP(
    np.prod(args.state_shape),
    output_dim=feature_dim,
    hidden_sizes=args.hidden_sizes[:-1],
    device=args.device,
)
action_dim = np.prod(args.action_shape)
icm_net = IntrinsicCuriosityModule(
    feature_net,
    feature_dim,
    action_dim,
    hidden_sizes=args.hidden_sizes[-1:],
    device=args.device,
).to(args.device)
icm_optim = torch.optim.Adam(icm_net.parameters(), lr=args.lr)
policy = ICMPolicy(
    policy=policy,
    model=icm_net,
    optim=icm_optim,
    action_space=env.action_space,
    lr_scale=args.lr_scale,
    reward_scale=args.reward_scale,
    forward_loss_weight=args.forward_loss_weight,
)

In [36]:

# collector
train_collector = Collector(
    policy,
    train_envs,
    VectorReplayBuffer(args.buffer_size, len(train_envs)),
)
test_collector = Collector(policy, test_envs)


In [37]:
# trainer
result = OnpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=args.epoch,
    step_per_epoch=args.step_per_epoch,
    repeat_per_collect=args.repeat_per_collect,
    episode_per_test=args.test_num,
    batch_size=args.batch_size,
    step_per_collect=args.step_per_collect,
).run()

Epoch #1: 50001it [00:21, 2323.57it/s, env_step=50000, gradient_step=800, len=216, n/ep=0, n/st=2000, rew=-61.36]                             


Epoch #1: test_reward: -173.421078 ± 39.062908, best_reward: -173.421078 ± 39.062908 in #1


Epoch #2: 50001it [00:21, 2300.87it/s, env_step=100000, gradient_step=1600, len=770, n/ep=0, n/st=2000, rew=41.51]                           


Epoch #2: test_reward: -25.458615 ± 20.297596, best_reward: -25.458615 ± 20.297596 in #2


Epoch #3: 50001it [00:22, 2222.29it/s, env_step=150000, gradient_step=2400, len=677, n/ep=4, n/st=2000, rew=78.27]                             


...

Epoch #8: test_reward: 259.509690 ± 41.484792, best_reward: 262.878976 ± 35.652734 in #6


Epoch #9: 50001it [00:21, 2318.57it/s, env_step=450000, gradient_step=7200, len=223, n/ep=8, n/st=2000, rew=254.75]                            


Epoch #9: test_reward: 275.918579 ± 16.502602, best_reward: 275.918579 ± 16.502602 in #9


Epoch #10: 50001it [00:23, 2140.37it/s, env_step=500000, gradient_step=8000, len=225, n/ep=6, n/st=2000, rew=236.83]                            


Epoch #10: test_reward: 235.407737 ± 80.287550, best_reward: 275.918579 ± 16.502602 in #9


In [40]:
env = gym.make(args.task,render_mode="human")
policy.eval()
test_num=3
for _ in range(test_num):
    collector = Collector(policy, env)
    result = collector.collect(n_episode=1, render=args.render)
    print(f"Final reward: {result.returns_stat.mean}, length: {result.lens_stat.mean}")
env.close()

Final reward: 257.96853752617494, length: 255.0
Final reward: 51.20673710968089, length: 1000.0
Final reward: 243.51113859610987, length: 260.0
